<a href="https://colab.research.google.com/github/YOKESH-P-S/Machine-Learning-Projects/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import os
import cv2
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.layers import LeakyReLU, Activation, ELU, PReLU

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize image to 128x128 pixels
            img = img.flatten()  # Flatten image into 1D array
            images.append(img)
    return np.array(images)

def inaccuracy(x):
    return (200 - x) * 0.5

def accuracy(x):
    return x * 0.5

def results(nc, nd):
    print("Number of cats in first 200 results:", nc)
    print("Number of dogs in next 200 results:", nd)
    print("Percentage accuracy for cat_test data:", accuracy(nc), "%")
    print("Percentage accuracy for dog_test data:", accuracy(nd), "%")
    print("Percentage inaccuracy for cat_test data:", inaccuracy(nc), "%")
    print("Percentage inaccuracy for dog_test data:", inaccuracy(nd), "%")
    print("\n")

cat_data = load_images_from_folder('cat')
dog_data = load_images_from_folder('dog')

num_images = min(len(cat_data), len(dog_data))
cat_data = cat_data[:num_images]
dog_data = dog_data[:num_images]

cat_train, cat_test = train_test_split(cat_data, test_size=0.2, random_state=42)
dog_train, dog_test = train_test_split(dog_data, test_size=0.2, random_state=42)

y_train = np.concatenate((np.ones(len(cat_train)), np.zeros(len(dog_train))))
y_test = np.concatenate((np.ones(len(cat_test)), np.zeros(len(dog_test))))

train_data = np.vstack((cat_train, dog_train))
test_data = np.vstack((cat_test, dog_test))

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

activation_functions = [
    'logistic',
    'relu',
    'tanh',
    LeakyReLU(alpha=0.01),
    ELU(alpha=1.0),
    Activation('swish'),
    PReLU()
]

class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}


for activation_function in activation_functions:
  if activation_function=="relu": #Custom Activation function
     if isinstance(activation_function, str):
       clf = MLPClassifier(hidden_layer_sizes=(500, 500), activation='relu', solver='adam', max_iter=10000)
     else:
       clf = MLPClassifier(hidden_layer_sizes=(500, 500), activation="tanh", solver='adam', max_iter=10000)
       clf.activation = "tanh"
     clf.fit(train_data, y_train)
     y_result = clf.predict(test_data)

nc = np.sum(y_result[:200] > 0.5)
nd = np.sum(y_result[200:] < 0.5)

print(f"Results for activation function: {activation_function}")
results(nc, nd)


Results for activation function: <keras.src.layers.activation.prelu.PReLU object at 0x7b7677d41bd0>
Number of cats in first 200 results: 117
Number of dogs in next 200 results: 111
Percentage accuracy for cat_test data: 58.5 %
Percentage accuracy for dog_test data: 55.5 %
Percentage inaccuracy for cat_test data: 41.5 %
Percentage inaccuracy for dog_test data: 44.5 %




In [166]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_images_from_folder(folder):

    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
    return np.array(images)

cat_data = load_images_from_folder('cat')
dog_data = load_images_from_folder('dog')

num_images = min(len(cat_data), len(dog_data))
cat_data = cat_data[:num_images]
dog_data = dog_data[:num_images]

split_point = int(0.8 * num_images)
cat_train, cat_test = cat_data[:split_point], cat_data[split_point:]
dog_train, dog_test = dog_data[:split_point], dog_data[split_point:]

train_data = np.vstack((cat_train, dog_train))
train_labels = np.array([0] * split_point + [1] * split_point)

test_data = np.vstack((cat_test, dog_test))
test_labels = np.array([0] * (num_images - split_point) + [1] * (num_images - split_point))

scaler = StandardScaler()
train_data_reshaped = train_data.reshape(train_data.shape[0], -1)
train_data_scaled = scaler.fit_transform(train_data_reshaped)

test_data_reshaped = test_data.reshape(test_data.shape[0], -1)
test_data_scaled = scaler.transform(test_data_reshaped)

train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

train_data = train_data.reshape(-1, 128, 128, 1)
test_data = test_data.reshape(-1, 128, 128, 1)

model = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
  MaxPooling2D((2, 2)),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

history = model.fit(datagen.flow(train_data, train_labels, batch_size=32),
                    epochs=5,
                    validation_data=(test_data, test_labels))

test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

print('Train Loss:', history.history['loss'][-1])
print('Train Accuracy:', history.history['accuracy'])

predictions = model.predict(test_data)
print(predictions.T)
predicted_labels = (predictions > 0.5).astype(int)
print(predicted_labels.T)

accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Accuracy on test set (manual calculation): {accuracy}")


Epoch 1/5
50/50 [==============================] - 44s 857ms/step - loss: 0.7227 - accuracy: 0.5075 - val_loss: 0.6905 - val_accuracy: 0.5100
Epoch 2/5
50/50 [==============================] - 43s 859ms/step - loss: 0.6855 - accuracy: 0.5575 - val_loss: 0.6862 - val_accuracy: 0.5150
Epoch 3/5
50/50 [==============================] - 44s 886ms/step - loss: 0.6833 - accuracy: 0.5781 - val_loss: 0.6677 - val_accuracy: 0.6325
Epoch 4/5
50/50 [==============================] - 43s 870ms/step - loss: 0.6801 - accuracy: 0.5788 - val_loss: 0.6707 - val_accuracy: 0.6150
Epoch 5/5
13/13 [==============================] - 2s 177ms/step - loss: 0.6725 - accuracy: 0.5550
Test accuracy: 0.5550000071525574
Test loss: 0.672507643699646
Train Loss: 0.6719188690185547
Train Accuracy: [0.5074999928474426, 0.5575000047683716, 0.578125, 0.5787500143051147, 0.590624988079071]
13/13 [==============================] - 2s 171ms/step
[[0.6696464  0.54433864 0.6457978  0.6352608  0.31954825 0.43573073
  0.400490